In [3]:
import math
import torch
from d2l import torch as d2l
from torch import nn

In [4]:
num_hiddens , num_heads = 100 , 5
attention = d2l.MultiHeadAttention(num_hiddens , num_hiddens , num_hiddens , num_hiddens , num_heads , 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [5]:
# self_attention 就是自己和自己也要做注意力计算
batch_size , num_queries = 2 , 4
vaild_lens = torch.tensor([3 , 2])
X = torch.ones(batch_size , num_queries , num_hiddens)
# 这里的输出就是自己
attention(X , X , X , vaild_lens).shape

torch.Size([2, 4, 100])

In [4]:
# 位置编码
class PositionalEncoding(nn.Module):
    """位置编码"""
    # max_len应该是输入序列的的长度
    def __init__(self, num_hiddens, dropout, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        # 创建一个足够长的P
        self.P = torch.zeros((1, max_len, num_hiddens))
        X = torch.arange(max_len, dtype=torch.float32).reshape(
            -1, 1) / torch.pow(10000, torch.arange(
            0, num_hiddens, 2, dtype=torch.float32) / num_hiddens)
        self.P[:, :, 0::2] = torch.sin(X)
        self.P[:, :, 1::2] = torch.cos(X)

    def forward(self, X):
        X = X + self.P[:, :X.shape[1], :].to(X.device)
        return self.dropout(X)

In [13]:
# encoding_dim表示每个token扩充的维度大小
# num_steps表示输入序列的长度大小
import pandas as pd
encoding_dim, num_steps = 64, 16
pos_encoding = PositionalEncoding(encoding_dim, 0)
pos_encoding.eval()
X = pos_encoding(torch.zeros((1, num_steps, encoding_dim)))
pd.DataFrame(X[0].detach().numpy())

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
1,0.841471,0.540302,0.681561,0.731761,0.533168,0.846009,0.409309,0.912396,0.310984,0.950415,...,0.000422,1.000000,0.000316,1.000000,0.000237,1.000000,0.000178,1.000000,0.000133,1.000000
2,0.909297,-0.416147,0.997480,0.070948,0.902131,0.431463,0.746904,0.664932,0.591127,0.806578,...,0.000843,1.000000,0.000632,1.000000,0.000474,1.000000,0.000356,1.000000,0.000267,1.000000
3,0.141120,-0.989992,0.778273,-0.627926,0.993253,-0.115966,0.953634,0.300967,0.812649,0.582754,...,0.001265,0.999999,0.000949,1.000000,0.000711,1.000000,0.000533,1.000000,0.000400,1.000000
4,-0.756802,-0.653644,0.141539,-0.989933,0.778472,-0.627680,0.993281,-0.115730,0.953581,0.301137,...,0.001687,0.999999,0.001265,0.999999,0.000949,1.000000,0.000711,1.000000,0.000533,1.000000
5,-0.958924,0.283662,-0.571127,-0.820862,0.323935,-0.946079,0.858896,-0.512150,0.999947,-0.010342,...,0.002108,0.999998,0.001581,0.999999,0.001186,0.999999,0.000889,1.000000,0.000667,1.000000
6,-0.279415,0.960170,-0.977396,-0.211417,-0.230368,-0.973104,0.574026,-0.818837,0.947148,-0.320796,...,0.002530,0.999997,0.001897,0.999998,0.001423,0.999999,0.001067,0.999999,0.000800,1.000000
7,0.656987,0.753902,-0.859313,0.511449,-0.713721,-0.700430,0.188581,-0.982058,0.800422,-0.599437,...,0.002952,0.999996,0.002214,0.999998,0.001660,0.999999,0.001245,0.999999,0.000933,1.000000
8,0.989358,-0.145500,-0.280228,0.959933,-0.977262,-0.212036,-0.229904,-0.973213,0.574318,-0.818632,...,0.003374,0.999994,0.002530,0.999997,0.001897,0.999998,0.001423,0.999999,0.001067,0.999999
9,0.412118,-0.911130,0.449194,0.893434,-0.939824,0.341660,-0.608109,-0.793854,0.291259,-0.956644,...,0.003795,0.999993,0.002846,0.999996,0.002134,0.999998,0.001600,0.999999,0.001200,0.999999


In [10]:
P = pos_encoding.P[:, :X.shape[1], :]
pd.DataFrame(P[0].detach().numpy(), index=range(num_steps))
# d2l.plot(torch.arange(num_steps), P[0, :, 6:10].T, xlabel='Row (position)',
#          figsize=(6, 2.5), legend=["Col %d" % d for d in torch.arange(6, 10)])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
1,0.841471,0.540302,0.310984,0.950415,0.099833,0.995004,0.031618,0.999500,0.010000,0.999950,0.003162,0.999995,0.001000,1.000000,0.000316,1.000000
2,0.909297,-0.416147,0.591127,0.806578,0.198669,0.980067,0.063203,0.998001,0.019999,0.999800,0.006325,0.999980,0.002000,0.999998,0.000632,1.000000
3,0.141120,-0.989992,0.812649,0.582754,0.295520,0.955337,0.094726,0.995503,0.029995,0.999550,0.009487,0.999955,0.003000,0.999996,0.000949,1.000000
4,-0.756802,-0.653644,0.953581,0.301137,0.389418,0.921061,0.126154,0.992011,0.039989,0.999200,0.012649,0.999920,0.004000,0.999992,0.001265,0.999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.636738,-0.771080,-0.190938,0.981602,-0.373877,0.927478,0.956817,-0.290690,0.556361,0.830941,0.185494,0.982645,0.058966,0.998260,0.018656,0.999826
60,-0.304811,-0.952413,0.123791,0.992308,-0.279415,0.960170,0.947148,-0.320796,0.564642,0.825336,0.188600,0.982054,0.059964,0.998201,0.018973,0.999820
61,-0.966118,-0.258102,0.426245,0.904608,-0.182163,0.983268,0.936532,-0.350583,0.572867,0.819648,0.191705,0.981453,0.060962,0.998140,0.019289,0.999814
62,-0.739181,0.673507,0.686428,0.727198,-0.083090,0.996542,0.924979,-0.380018,0.581035,0.813878,0.194808,0.980842,0.061960,0.998079,0.019605,0.999808
